In [1]:
#import the libraries
import pandas as pd
import pickle
import networkx as nx

In [2]:
#load the csv file and view its contents
df = pd.read_csv('all_flights.csv')
#display the dataframe
df

,flight_no,airline,departure_time,origin,arrival_time,destination,type,capacity
0,DL2313,Delta,7:00,LAX,8:35,SFO,Boeing 737-800,165
1,DL1806,Delta,9:05,LAX,10:38,SFO,Airbus A319,128
2,DL1327,Delta,12:58,LAX,14:27,SFO,Airbus A319,128
3,DL2757,Delta,15:18,LAX,16:44,SFO,Airbus A320,150
4,DL2803,Delta,17:50,LAX,19:14,SFO,Airbus A320,150
...,...,...,...,...,...,...,...,...
543,2264,American Airlines,13:30,BOS,15:33,ORD,321,185
544,1109,American Airlines,19:14,BOS,21:22,ORD,738,165
545,197,American Airlines,6:01,BOS,7:26,JFK,738,165
546,1283,American Airlines,9:33,BOS,10:59,JFK,32B,185


In [3]:
#Convert arrival_time to datetime object and round off to the nearest hour
df['arrival'] = (pd.to_datetime(df['arrival_time']).dt.round('H').dt.hour)
#Convert departure_time to datetime object and round off to the nearest hour
df['departure'] = (pd.to_datetime(df['departure_time']).dt.round('H').dt.hour)

In [4]:
#View the new dataframe
df

,flight_no,airline,departure_time,origin,arrival_time,destination,type,capacity,arrival,departure
0,DL2313,Delta,7:00,LAX,8:35,SFO,Boeing 737-800,165,9,7
1,DL1806,Delta,9:05,LAX,10:38,SFO,Airbus A319,128,11,9
2,DL1327,Delta,12:58,LAX,14:27,SFO,Airbus A319,128,14,13
3,DL2757,Delta,15:18,LAX,16:44,SFO,Airbus A320,150,17,15
4,DL2803,Delta,17:50,LAX,19:14,SFO,Airbus A320,150,19,18
...,...,...,...,...,...,...,...,...,...,...
543,2264,American Airlines,13:30,BOS,15:33,ORD,321,185,16,14
544,1109,American Airlines,19:14,BOS,21:22,ORD,738,165,21,19
545,197,American Airlines,6:01,BOS,7:26,JFK,738,165,7,6
546,1283,American Airlines,9:33,BOS,10:59,JFK,32B,185,11,10


In [5]:
#Replace 0 with 24 in the arrival column for easier comparisons
df.arrival = df.arrival.replace(0, 24)

In [6]:
#Storing all the airports in a list
ports = list(df['origin'].unique())
ports.append('JFK')

In [7]:
#Create a mapping dictionary to map the name of the airport with a number from 0 to 9
#For example, LAX becomes 0 and JFK becomes 9
mapping = dict(zip(ports, range(10)))

In [8]:
#Converting the string values of the origin to numbers based on the mapping
df['origin'] = df['origin'].apply(lambda x: mapping[x])
#Converting the string values of the origin to numbers based on the mapping
df['destination'] = df['destination'].apply(lambda x: mapping[x])

In [9]:
#View the new dataframe
df

,flight_no,airline,departure_time,origin,arrival_time,destination,type,capacity,arrival,departure
0,DL2313,Delta,7:00,0,8:35,1,Boeing 737-800,165,9,7
1,DL1806,Delta,9:05,0,10:38,1,Airbus A319,128,11,9
2,DL1327,Delta,12:58,0,14:27,1,Airbus A319,128,14,13
3,DL2757,Delta,15:18,0,16:44,1,Airbus A320,150,17,15
4,DL2803,Delta,17:50,0,19:14,1,Airbus A320,150,19,18
...,...,...,...,...,...,...,...,...,...,...
543,2264,American Airlines,13:30,6,15:33,8,321,185,16,14
544,1109,American Airlines,19:14,6,21:22,8,738,165,21,19
545,197,American Airlines,6:01,6,7:26,9,738,165,7,6
546,1283,American Airlines,9:33,6,10:59,9,32B,185,11,10


In [10]:
#Converting the port to a list of numbers based on the mapping
ports = list(mapping.values())
#View the new ports list
ports

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [11]:
#Convert dataframe to a Multi Directed Graph using NetworkX
#The edges contain the following columns: departure, arrival and capacity
G = nx.from_pandas_edgelist(df, source='origin', target='destination', edge_attr=['departure', 'arrival', 'capacity'], create_using=nx.MultiDiGraph())

In [12]:
#Global variables
#edgelist keeps track of all the edges that are visited
edgelist = []

#Function to get the first edge which has a nonzero capacity and satisfies the time constraints
def getedge(s, t, edges):
    #References the global edgelist to add an edge to the edgelist
    global edgelist
    for i, k in enumerate(edges):
        #Condition to check if the node is the source node, this ignores the time constraints
        if edges[k]['capacity'] > 0 and s==0:
            edgelist.append((edges[k], k, s, t))
            return True
        #Condition to check for both capacity and time constraints
        elif edges[k]['capacity'] > 0 and edgelist[-1][0]['arrival'] <= edges[k]['departure']:
            edgelist.append((edges[k], k, s, t))
            return True
    return False

#Checks if there any outgoing edges from LAX aka 0
def conn(G):
    if len(list(G.neighbors(0))) == 0:
        return False
    return True

#Function to perform Bread-First-Search on the graph
def BFS(G, s, t, parent):
    visited = [False] * len(ports)
    queue = []
    queue.append(s)
    visited[s] = True
    while queue:
        u = queue.pop(0)
        for ind in G.neighbors(u):
            #Finds the first unvisited node
            if visited[ind] == False:
                #Gets the edges from u to unvisited node
                edges = G.get_edge_data(u, ind)
                if(edges != None):
                    #Determines whether there is a feasible edge
                    if getedge(u, ind, edges):
                        queue.append(ind)
                        visited[ind] = True
                        parent[ind] = u
                    else:
                        continue
                else:
                    continue
    #Returns true if the sink is visited
    return True if visited[t] else False

#Function to perform Ford-Fulkerson on the graph
def FordFulkerson(G, source, sink): 
    #References global edgelist to retrieve all the visited edges
    global edgelist 
    parent = [-1] * len(ports)
    #Max flow is initially zero
    max_flow = 0
    
    #Perform BFS iteratively to find a path
    while conn(G) and BFS(G, source, sink, parent): 
        path_flow = float("Inf") 
        
        #New edgelist is created to filter out the required edges which formed the path
        elist = []
        #Filters out the required edges while simultaneously finding the minimum flow along the path
        s = sink
        while(s != source):
            for edge in edgelist:
                if edge[-1] == s and edge[-2] == parent[s]:
                    path_flow = min(path_flow, edge[0]['capacity'])
                    #Required edges along the path are added to elist
                    elist.append(edge)
            s = parent[s]
        
        #Minimum flow along the path is added maximum flow
        max_flow +=  path_flow 
        
        #Subtracts the minimum flow from each capacity on the path that was obtained (present in the elist)
        while(len(elist)>0):
            j = len(elist)-1
            #Edges in the elist are represented as a tuple consisting of (edgedict, key, source, destination)
            so = elist[j][2]
            dt = elist[j][3]
            k = elist[j][1]
            #Update the capacity of each edge by subtracting minimum flow
            G[so][dt][k]['capacity'] = G[so][dt][k]['capacity'] - path_flow
            #Checks if capacity of the edge is less or equal to zero
            if G[so][dt][k]['capacity'] <= 0:
                #Deletes the edge from the graph is its capacity is less than zero
                G.remove_edge(so, dt, k)
            #Removes the edge from the elist
            elist.remove(elist[j])
        #Empties the global edgelist for the next iteration of BFS
        edgelist = []
    
    #Returns the maximum flow
    return max_flow 
#Calls the FordFulkerson algorithm to calculate maximum capacity of the NAS
max_capacity = FordFulkerson(G, 0, 9)
print("The maximum capacity of the NAS is: ", max_capacity)

The maximum capacity of the NAS is:  6571
